## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

## Here We're Going to Build a Manual Machine Learning Solution, to engineer the features in the dataset and predict who is more likely to Survive the catastrophe.

## Imports

In [ ]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [ ]:
# load the data - it is available open source and online
#data = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
data = pd.read_csv('../Data/titanic.csv')

# display data
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# replace question marks by NaN values
data = data.replace('?', np.nan)

In [ ]:
data['Cabin'].unique()

In [ ]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan
    
data['Cabin'] = data['Cabin'].apply(get_first_cabin)

In [ ]:
# extracts the title (Mr, Ms, etc) from the name variable
def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    elif re.search('Dr', line):
        return 'Dr'
    else:
        return 'Other'
    
data['Title'] = data['Name'].apply(get_title)

In [ ]:
data.info()

In [ ]:
# drop unnecessary variables
data.drop(labels=['PassengerId','Name','Ticket'], axis=1, inplace=True)

# display data
data.head()

In [ ]:
# save the data set

#data.to_csv('titanic.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [ ]:
target = 'Survived'

In [ ]:
vars_num = [c for c in data.columns if data[c].dtypes!='O' and c!=target]

vars_cat = [c for c in data.columns if data[c].dtypes=='O']

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

In [ ]:
vars_num

### Find missing values in variables

In [ ]:
# first in numerical variables

data[vars_num].isnull().mean()

In [ ]:
# now in categorical variables

data[vars_cat].isnull().mean()

### Determine cardinality of categorical variables

In [ ]:
data[vars_cat].nunique()

### Determine the distribution of numerical variables

In [ ]:
data[vars_num].hist(bins=30, figsize=(10,10))
plt.show()

## Separate data into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Survived', axis=1),  # predictors
    data['Survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

In [ ]:
X_train.columns

In [ ]:
X_test.columns

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [ ]:
X_train['Cabin'] = X_train['Cabin'].str[0] # captures the first letter
X_test['Cabin'] = X_test['Cabin'].str[0] # captures the first letter

X_train['Cabin'].unique()

In [ ]:
X_test['Cabin'].unique()

### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [ ]:
for var in ['Age', 'Fare']:

    # add missing indicator
    X_train[var+'_NA'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var+'_NA'] = np.where(X_test[var].isnull(), 1, 0)

    # replace NaN by median
    median_val = X_train[var].median()

    X_train[var] = X_train[var].fillna(median_val)
    X_test[var] = X_test[var].fillna(median_val)

X_train[['Age', 'Fare']].isnull().sum()

In [ ]:
X_test

### Replace Missing data in categorical variables with the string **Missing**

In [ ]:
X_train[vars_cat] = X_train[vars_cat].fillna('Missing')
X_test[vars_cat] = X_test[vars_cat].fillna('Missing')

In [ ]:
X_train.isnull().sum()

In [ ]:
X_test.isnull().sum()

### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [ ]:
def find_frequent_labels(df, var, rare_perc):
    
    # function finds the labels that are shared by more than
    # a certain % of the passengers in the dataset
    
    df = df.copy()
    
    tmp = df.groupby(var)[var].count() / len(df)
    
    return tmp[tmp > rare_perc].index


for var in vars_cat:
    
    # find the frequent categories
    frequent_ls = find_frequent_labels(X_train, var, 0.05)
    
    # replace rare categories by the string "Rare"
    X_train[var] = np.where(X_train[var].isin(
        frequent_ls), X_train[var], 'Rare')
    
    X_test[var] = np.where(X_test[var].isin(
        frequent_ls), X_test[var], 'Rare')

In [ ]:
X_train[vars_cat].nunique()

In [ ]:
X_test[vars_cat].nunique()

### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [ ]:
for var in vars_cat:
    
    # to create the binary variables, we use get_dummies from pandas
    
    X_train = pd.concat([X_train,
                         pd.get_dummies(X_train[var], prefix=var, drop_first=True)
                         ], axis=1)
    
    X_test = pd.concat([X_test,
                        pd.get_dummies(X_test[var], prefix=var, drop_first=True)
                        ], axis=1)
    

X_train.drop(labels=vars_cat, axis=1, inplace=True)
X_test.drop(labels=vars_cat, axis=1, inplace=True)

X_train.shape, X_test.shape

In [ ]:
# Note that we have one less column in the test set
# this is because we had 1 less category in embarked.

# we need to add that category manually to the test set

X_train.head()

In [ ]:
X_test.head()

In [ ]:
# we add 0 as values for all the observations, as Rare
# was not present in the test set

X_test['Embarked_Rare'] = 0

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# Note that now embarked_Rare will be at the end of the test set
# so in order to pass the variables in the same order, we will
# create a variables variable:

variables = [c  for c in X_train.columns]

variables

### Scale the variables

- Use the standard scaler from Scikit-learn

In [ ]:
# create scaler
scaler = StandardScaler()

#  fit  the scaler to the train set
scaler.fit(X_train[variables]) 

# transform the train and test set
X_train = scaler.transform(X_train[variables])

X_test = scaler.transform(X_test[variables])

In [ ]:
X_test

## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [ ]:
# set up the model
# remember to set the random_state / seed

model = LogisticRegression(C=0.005, random_state=0)

# train the model
model.fit(X_train, y_train)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [ ]:
# make predictions for test set
class_ = model.predict(X_train)
pred = model.predict_proba(X_train)[:,1]

# determine mse and rmse
print('train roc-auc: {}'.format(roc_auc_score(y_train, pred)))
print('train accuracy: {}'.format(accuracy_score(y_train, class_)))
print()

# make predictions for test set
class_ = model.predict(X_test)
pred = model.predict_proba(X_test)[:,1]

# determine mse and rmse
print('test roc-auc: {}'.format(roc_auc_score(y_test, pred)))
print('test accuracy: {}'.format(accuracy_score(y_test, class_)))
print()

# List of problems if we want to make future predictions
1- Re-write a lot of code ==> repetitive

2- Hard coded a lot of parameters ==> if these change we need to re-write them again.

3- We may engineer a lot of variables that we actually do not need for the model.

4- Additional variables may present missing data, we do not know what to do with them!

**We can minimize these hurdles by using Open-source. That's what we will learn!**